# Imports


In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.23.5


In [7]:
pip install tensorflow

     |████████████████████████████████| 216.2 MB 11 kB/s  eta 0:00:011    |██████▏                         | 41.7 MB 57.7 MB/s eta 0:00:04     |███████▋                        | 51.5 MB 57.7 MB/s eta 0:00:03
     |████████████████████████████████| 19.8 MB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 440 kB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 394 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 25.0 MB/s eta 0:00:01
     |████████████████████████████████| 135 kB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 26.5 MB 26.7 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 57 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 805 kB/s  eta 0:00:01
     |████████████████████████████████| 1.7 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.

In [8]:
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed

# Loading

In [ ]:
# Load the second uploaded notebook

notebook_path_2 = "./Parallel CNN-Based Transient Stability Analysis.ipynb"
with open(notebook_path_2, 'r', encoding='utf-8') as f:
    notebook_2 = nbformat.read(f, as_version=4)

# Extract code cells from the second notebook
code_cells_2 = [cell['source'] for cell in notebook_2.cells if cell.cell_type == 'code']
code_cells_2[:3]  # Show first few code cells to understand the structure



# Scaling Test

In [ ]:
# Scaling Test (Data Size Only)

def build_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Split data once to reuse same test set
X_train_full, X_test, y_train_full, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

fractions = [0.1, 0.25, 0.5, 0.75, 1.0]
results = []

for frac in fractions:
    size = int(len(X_train_full) * frac)
    X_train = X_train_full[:size]
    y_train = y_train_full[:size]

    model = build_model(X_train.shape[1:])

    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    duration = time.time() - start_time

    y_pred = model.predict(X_test).flatten() > 0.5
    acc = accuracy_score(y_test, y_pred)

    results.append((frac, size, duration, acc))
    print(f"Data Fraction: {frac}, Train Size: {size}, Time: {duration:.2f}s, Accuracy: {acc:.4f}")




In [ ]:
# Display results as DataFrame
df_results = pd.DataFrame(results, columns=["Fraction", "Train Size", "Time (s)", "Accuracy"])
print(df_results)


In [ ]:
# Plot
df_results.plot(x="Train Size", y=["Time (s)", "Accuracy"], marker='o', title="Scaling Test Results")
plt.grid(True)
plt.show()
